# Exploración de la distribución normal bivariada

Como estudiante de educación matemática, me propuse crear esta libreta para repasar qué significa modelar fenómenos con una distribución normal en dos variables. La meta es mezclar una explicación amigable con algunos gráficos que podamos reutilizar en clase.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from scipy.stats import multivariate_normal

plt.style.use("seaborn-v0_8-whitegrid")
np.random.seed(42)

## ¿Qué datos necesito?

Para describir una normal bivariada basta con un vector de medias $\mu$ y una matriz de covarianzas $\Sigma$. En esta primera parte usaré:

$$
\mu = \begin{pmatrix} 1 \\ 2 \end{pmatrix}, \quad
\Sigma = \begin{pmatrix} 2 & 1.2 \\ 1.2 & 1.5 \end{pmatrix}
$$

La covarianza positiva (1.2) nos adelanta que las variables tienden a crecer juntas.

In [ ]:
mean = np.array([1, 2])
cov = np.array([[2, 1.2], [1.2, 1.5]])

x = np.linspace(-3, 5, 200)
y = np.linspace(-2, 6, 200)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

rv = multivariate_normal(mean, cov)
Z = rv.pdf(pos)

fig, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(X, Y, Z, levels=20, cmap="viridis")
ax.set_title("Densidad conjunta de la normal bivariada")
ax.set_xlabel("Variable X")
ax.set_ylabel("Variable Y")
fig.colorbar(contour, ax=ax, label="f(x, y)")
plt.show()

La gráfica de contorno nos deja ver la forma elíptica típica de la normal bivariada. La dirección principal de la elipse indica cómo se relacionan las variables: una inclinación positiva implica correlación positiva.

In [ ]:
sample = rv.rvs(size=500)
x_samples, y_samples = sample[:, 0], sample[:, 1]

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(x_samples, y_samples, alpha=0.6, edgecolor="k", s=40)
ax.set_title("Simulación de 500 pares (X, Y)")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.axvline(mean[0], color="C3", linestyle="--", label="Media X")
ax.axhline(mean[1], color="C1", linestyle="--", label="Media Y")
ax.legend()
plt.show()

El diagrama de dispersión confirma lo que se observó en la densidad: las parejas de datos se acomodan alrededor de las medias, y la nube se inclina porque ambas variables están positivamente correlacionadas.

In [ ]:
def draw_covariance_ellipse(ax, mean, cov, n_std=2.0, **kwargs):
    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))
    width, height = 2 * n_std * np.sqrt(vals)
    ellipse = Ellipse(mean, width, height, theta, **kwargs)
    ax.add_patch(ellipse)

fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharex=True, sharey=True)
correlations = [0, 0.6, -0.8]

for ax, rho in zip(axs, correlations):
    cov_matrix = np.array([[1, rho], [rho, 1]])
    mvn = multivariate_normal([0, 0], cov_matrix)
    data = mvn.rvs(size=400)

    ax.scatter(data[:, 0], data[:, 1], alpha=0.5, s=30)
    draw_covariance_ellipse(ax, [0, 0], cov_matrix, edgecolor="C3", facecolor="none", linewidth=2)
    ax.set_title(f"Correlación $\\rho = {rho}$")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

fig.suptitle("Comparación de correlaciones en la normal bivariada", fontsize=16)
plt.tight_layout()
plt.show()

Cuando la correlación es cero, la elipse queda horizontal y vertical, indicando independencia. Con correlación positiva, la elipse se inclina hacia el primer cuadrante. Si la correlación es negativa, se inclina hacia el cuarto cuadrante y la nube se estira en sentido opuesto.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401, requerido para habilitar 3D

fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(X, Y, Z, cmap="viridis", linewidth=0, antialiased=True, alpha=0.9)
ax.set_title("Superficie de densidad de la normal bivariada")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("f(x, y)")
plt.tight_layout()
plt.show()

La superficie 3D es útil para mostrar que la densidad conjunta alcanza su máximo alrededor de la media y decrece exponencialmente en todas las direcciones. Este tipo de visualización puede motivar discusiones sobre probabilidades conjuntas y regiones de mayor concentración de datos.

## Reflexión final

Este recorrido me ayuda a visualizar cómo cambian las probabilidades cuando modificamos la correlación entre las variables. En una clase de estadística educativa podríamos pedir al alumnado que manipule la covarianza y observe los cambios, conectando el álgebra matricial con fenómenos reales como el vínculo entre horas de estudio y calificaciones.